# Practicioners Seminar

In [7]:
import numpy as np
import pandas as pd
import seaborn as sns
import os

## Task 2

### A correlation heatmap for currencies and equity markets

In [20]:
currencies = "FX_CHF_ALL_1971-2020.xlsx"
df = pd.read_excel(currencies, header=0, index_col=0, sheet_name="Sheet1")
print(df)

            CHFUSD  CHFEUR  CHFGBP  CHFAUD   CHFJPY  CHFCAD
Date                                                       
2020-12-22  1.1301  0.9224  0.8422  1.4941  116.814  1.4535
2020-12-21  1.1291  0.9219  0.8386  1.4880  116.655  1.4516
2020-12-18  1.1323  0.9235  0.8369  1.4857  116.924  1.4474
2020-12-17  1.1308  0.9216  0.8321  1.4830  116.543  1.4385
2020-12-16  1.1298  0.9261  0.8360  1.4911  116.852  1.4394
2020-12-15  1.1291  0.9290  0.8385  1.4937  117.050  1.4340
2020-12-14  1.1274  0.9284  0.8460  1.4964  117.326  1.4390
2020-12-11  1.1237  0.9276  0.8498  1.4913  116.895  1.4350
2020-12-10  1.1289  0.9297  0.8492  1.4979  117.645  1.4382
2020-12-09  1.1243  0.9307  0.8389  1.5100  117.197  1.4411
2020-12-08  1.1249  0.9292  0.8422  1.5174  117.151  1.4420
2020-12-07  1.1224  0.9270  0.8386  1.5128  116.839  1.4367
2020-12-04  1.1212  0.9251  0.8342  1.5101  116.728  1.4333
2020-12-03  1.1228  0.9239  0.8345  1.5085  116.553  1.4431
2020-12-02  1.1177  0.9225  0.8362  1.50

In [22]:

return_cor = df.pct_change().corr()
#sns.heatmap(return_cor, xticks=return_cor.columns.values, yticks=return_cor.columns.values)

In [24]:
print(return_cor)

          CHFUSD    CHFEUR    CHFGBP    CHFAUD    CHFJPY    CHFCAD
CHFUSD  1.000000  0.155969  0.201779  0.111982  0.590179  0.204282
CHFEUR  0.155969  1.000000  0.181164  0.157604  0.090658  0.185282
CHFGBP  0.201779  0.181164  1.000000  0.172515  0.115671  0.199382
CHFAUD  0.111982  0.157604  0.172515  1.000000  0.046299  0.174503
CHFJPY  0.590179  0.090658  0.115671  0.046299  1.000000  0.116190
CHFCAD  0.204282  0.185282  0.199382  0.174503  0.116190  1.000000
